### Packages

In [21]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [22]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [23]:
df_comp = raw_data.copy()

In [24]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [25]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

### Creating Returns

In [26]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [27]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [28]:
from pmdarima.arima import auto_arima

In [29]:
model_auto = auto_arima(df.ret_ftse[1:])

In [30]:
model_auto

ARIMA(maxiter=50, method='lbfgs', order=(2, 0, 5), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=False, trend=None,
      with_intercept=True)

In [31]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(2, 0, 5)   Log Likelihood               -7883.803
Date:                Sun, 12 Apr 2020   AIC                          15785.606
Time:                        02:29:53   BIC                          15844.295
Sample:                             0   HQIC                         15806.171
                               - 5019                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0314      0.024      1.307      0.191      -0.016       0.078
ar.L1          0.1763      0.039      4.556      0.000       0.100       0.252
ar.L2         -0.8137      0.035    -23.113      0.000      -0.883      -0.745
ma.L1         -0.2004      0.038     -5.259      0.000      -0.275      -0.126
ma.L2          0.7661      0.037     20.538      0.000       0.693       0.839
ma.L3         -0.0952      0.012     -8.237      0.000      -0.118      -0.073
ma.L4          0.0108      0.009      1.190      0.234      -0.007       0.029
ma.L5         -0.1110      0.009    -12.939      0.000      -0.128      -0.094
sigma2         1.3549      0.014     94.027      0.000       1.327       1.383
===================================================================================
Ljung-Box (Q):                       69.74   Jarque-Bera (JB):              6583.04
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.18
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.60
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [32]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' (out of bag)
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [33]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 6275
Model:             SARIMAX(2, 0, 2)x(1, 0, [1], 5)   Log Likelihood               -6340.150
Date:                             Sun, 12 Apr 2020   AIC                          12704.301
Time:                                     02:32:06   BIC                          12785.233
Sample:                                 01-10-1994   HQIC                         12732.342
                                      - 01-26-2018                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0043      0.009     -0.499      0.618      -0.021       0.013
drift      -1.452e-06   2.72e-06     -0.534      0.593   -6.78e-06    3.88e-06
ret_spx        0.0958      0.006     16.945      0.000       0.085       0.107
ret_dax        0.5582      0.005    113.711      0.000       0.549       0.568
ret_nikkei     0.0702      0.004     16.545      0.000       0.062       0.079
ar.L1         -0.1932      0.101     -1.919      0.055      -0.391       0.004
ar.L2          0.5083      0.059      8.640      0.000       0.393       0.624
ma.L1          0.0863      0.101      0.855      0.393      -0.112       0.284
ma.L2         -0.5368      0.056     -9.533      0.000      -0.647      -0.426
ar.S.L5        0.0705      0.339      0.208      0.835      -0.595       0.736
ma.S.L5       -0.0987      0.339     -0.291      0.771      -0.764       0.567
sigma2         0.4405      0.004     99.114      0.000       0.432       0.449
===================================================================================
Ljung-Box (Q):                       84.25   Jarque-Bera (JB):             15316.47
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.55   Skew:                             0.25
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""